In [ ]:
# Part 1 sim check
# pull random cards from a deck

from random import random, choice


def new_card(cnts):
    # given a dictionary of counts return a suit of card with prob proportial to the suits
    x = random()
    total = sum(cnts.values())
    P = 0
    if total == 0:
        return None
    for t, cnt in cnts.items():
        P += cnt / total
        if x <= P:
            cnts[t] -= 1
            return t


def check1(cards):
    N = len(cards)
    c1 = cards[0]
    same = all(c == c1 for c in cards)
    diff = len(set(cards)) == N
    return same or diff


def sim_game(num_trials=100000):

    def trial():
        deck = {1: 14, 2: 14, 3: 14}
        total = 42
        cards = [new_card(deck) for _ in range(3)]

        # print(cards)
        # print(deck)
        if check1(cards):
            return 1
        return 0

    return sum(trial() for _ in range(num_trials)) / num_trials


print(sim_game())
print(f"{137/410}")

0.33588
0.33414634146341465


In [ ]:
# now expected value with wildcards
from functools import cache
from fractions import Fraction


@cache
def E(t1, t2, t3, w):
    # all of one type
    if max(t1, t2, t3) + w == 3:
        return Fraction(0)

    # one of each type
    if w == 2 and (t1 + t2 + t3 >= 1):
        return 0
    if w == 1 and ((t1 > 0 and t2 > 0) or (t1 > 0 and t3 > 0) or (t2 > 0 and t3 > 0)):
        return 0
    if w == 0 and (t1 > 0 and t2 > 0 and t3 > 0):
        return 0
    # expected turns from current state
    N = 44 - t1 - t2 - t3 - w
    p1, p2, p3, pw = (
        Fraction(14 - t1, N),
        Fraction(14 - t2, N),
        Fraction(14 - t3, N),
        Fraction(2 - w, N),
    )
    return (
        1
        + p1 * E(t1 + 1, t2, t3, w)
        + p2 * E(t1, t2 + 1, t3, w)
        + p3 * E(t1, t2, t3 + 1, w)
        + pw * E(t1, t2, t3, w + 1)
    )


ans = E(0, 0, 0, 0)
print(f"The expected number of turns is {ans} which is around ~ {ans:.4f}")

The expected number of turns is 72921/19393 which is around ~ 3.7602


In [12]:
# simulate
from collections import defaultdict


def check2(cnts):
    t1, t2, t3, w = [cnts[i] for i in range(1, 5)]
    if max(t1, t2, t3) + w == 3:
        return True

    # one of each type
    if w == 2 and (t1 + t2 + t3 >= 1):
        return True
    if w == 1 and ((t1 > 0 and t2 > 0) or (t1 > 0 and t3 > 0) or (t2 > 0 and t3 > 0)):
        return True
    if w == 0 and (t1 > 0 and t2 > 0 and t3 > 0):
        return True
    return False


def cards_needed(cards):
    N = len(cards)
    cnt = 0

    seen = defaultdict(int)
    for c in cards:
        cnt += 1
        seen[c] += 1
        if check2(seen):
            break
    return cnt


def sim_E_game(num_trials=100000):

    def trial_E():
        deck = {1: 14, 2: 14, 3: 14, 4: 2}
        cards = [new_card(deck) for _ in range(6)]
        return cards_needed(cards)

    return sum(trial_E() for _ in range(num_trials)) / num_trials


print(f"The expected number of turns is {ans} which is around ~ {ans:.4f}")
print(sim_E_game())

The expected number of turns is 72921/19393 which is around ~ 3.7602
3.760032235528942
